### Introduction

We have deployed our model and it is successfully being used to determine the class of Iris flowers by botanists all over the world!

Enjoying our success and the data science we have been doing, has resulted in us spending more time learning more about LinearSVC models.

One thing we noticed in the [LinearSVC documentation](https://scikit-learn.org/stable/modules/svm.html#svm-classification) was the following:

> Support Vector Machine algorithms are not scale invariant, so it is highly recommended to scale your data.

Oh. What does 'scale your data' mean?


### Scaling Data

There are many ways to scale data (see the [Appendix](#Appendix)).  A good starting point for learning about scalers is StandardScaler:

> Standardize features by removing the mean and scaling to unit variance.
>
> *Source: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html*

[TODO more coming soon ...]


In [6]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
    
iris = datasets.load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
std_slc = StandardScaler()
std_slc.fit(X_train)
X_train_std = std_slc.transform(X_train)
X_test_std = std_slc.transform(X_test)

print(X_train[0:1])
print(X_train_std[0:1])

[[7.6 3.  6.6 2.1]]
[[ 2.246205   -0.16335233  1.61145803  1.13551763]]


### Inconsistent Preprocessing


>  If ... data transforms are used when training a model, they also must be used on subsequent datasets, whether it’s test data or data in a production system. Otherwise, the feature space will change, and the model will not be able to perform effectively.
>
> https://scikit-learn.org/stable/common_pitfalls.html#inconsistent-preprocessing

[TODO] Let's apply this now to our scaled iris model ...



---
### Navigation

[Previous](./11_model_predictions.ipynb) | [Home](./00-README-FIRST.ipynb) | [Next](./98_outro.ipynb) notebook

---

<div class="alert-info">

### Appendix

Other Scikit-learn scalers:

- [sklearn.preprocessing.MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)
- [sklearn.preprocessing.MaxAbsScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MaxAbsScaler.html)
- [sklearn.preprocessing.StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)
- [sklearn.preprocessing.RobustScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html)
- [sklearn.preprocessing.Normalizer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html)
- [sklearn.preprocessing.QuantileTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.QuantileTransformer.html)
- [sklearn.preprocessing.PowerTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PowerTransformer.html)

</div>